In [1]:
import os
os.chdir('../')
%pwd

'C:\\Users\\VICTUS\\Documents\\Course Material\\Krish-Naik-End-to-End-ML-Project\\Chicken-Disease-Classifier-Project'

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_weights: str
    params_classes: int

In [3]:
from cnn_classifier.constants import *
from cnn_classifier.utils.common import read_yaml, create_directories


In [4]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        
        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_image_size=self.params.IMAGE_SIZE,
            params_learning_rate=self.params.LEARNING_RATE,
            params_include_top=self.params.INCLUDE_TOP,
            params_weights=self.params.WEIGHTS,
            params_classes=self.params.CLASSES
        )
        return prepare_base_model_config

In [20]:
import torch
import torchvision
import torch.nn as nn
from cnn_classifier import logger

class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config
    
    def get_base_model(self):
        self.model = torchvision.models.vgg16(pretrained=True)
        self.save_model(path=self.config.base_model_path, model=self.model)
    
    @staticmethod
    def _prepare_full_model(model, num_classes, learning_rate):            
        # Freeze the parameters of the model.
        for param in model.parameters():
            param.requires_grad = False
        
        n_inputs = model.classifier[6].in_features
        model.classifier[6] = nn.Linear(n_inputs, num_classes)        
        # check if CUDA is available
        use_cuda = torch.cuda.is_available()
        if use_cuda:
            model = model.cuda()
        return model
        
    def update_base_model(self):
        self.full_model = self._prepare_full_model(
            model=self.model,
            num_classes=self.config.params_classes,
            learning_rate=self.config.params_learning_rate
        )
        logger.info(self.full_model)
        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)
        
    @staticmethod
    def save_model(path: Path, model):
        torch.save(model.state_dict(), path)

In [22]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e

[2023-09-24 00:18:59,557]: INFO: common: yaml file: config\config.yaml loaded successfully
[2023-09-24 00:18:59,560]: INFO: common: yaml file: params.yaml loaded successfully
[2023-09-24 00:18:59,561]: INFO: common: created directory at: artifacts
[2023-09-24 00:18:59,563]: INFO: common: created directory at: artifacts/prepare_base_model
[2023-09-24 00:19:01,717]: INFO: 3822555531: VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  